<a href="https://colab.research.google.com/github/kimdonggyu2008/hufs_hackerthon/blob/main/test_dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch-optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
#library
import numpy as np
import os
import pickle #전처리된 데이터를 파일로 저장하거나 불러올 때 사용
import sys
import pandas as pd
import re
import cv2

#torch
import torch
import torch.cuda.amp as amp #Pytorch의 혼합 정밀도(Amp, Automatic, Mixed Precision) 기능 제공
from torch.utils.data.dataset import Dataset #데이터를 미니배치 단위로 불러오고, 셔플링 및 병렬 처리 지원
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

import torch.nn as nn
import torch.nn.functional as F

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau, MultiStepLR, OneCycleLR
'''
CosineAnnealingWarmRestarts: 코사인 함수 기반으로 학습률을 조절하며, 주기마다 학습률을 초기화
CosineAnnealingLR: 코사인 함수 형태로 학습률 감소시킴
ReduceLROnPlateau: 성능 향상이 없을 때 학습률 감소시킴
MultiStepLR: 지정된 에포크마다 학습률 변경
OneCycleLR: 한 사이클 내에서 학습률을 증가시키고 감소시키는 스케줄러
'''
import math
import torch
from torch.optim.optimizer import Optimizer, required #옵티마이저 필수 인자 지정

import torch_optimizer as optim

# utility
from collections import defaultdict #기본 딕셔너리를 생성
import itertools as it #효율적인 반복문 작성을 위해
import tqdm
import random
import matplotlib.pyplot as plt
from timeit import default_timer as timer

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# transformers
import transformers
from transformers import XLMPreTrainedModel, XLMRobertaModel, XLMRobertaConfig, XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, XLNetForSequenceClassification, XLMRobertaForSequenceClassification, XLMForSequenceClassification, RobertaForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [ ]:
# 하이퍼파라미터 설정 클래스
class args:
    # factor
    debug = False #디버그 모드
    amp = True # 자동 혼합 정밀도 사용 여부
    gpu ='0'

    epochs =10
    batch_size = 32
    weight_decay = 1e-6 #옵티마이저의 weight decay 값 설정
    n_fold = 3
    fold=3

    exp_name = '/content/drive/MyDrive/코딩공부/dacon_project/result' #실험 결과 저장 폴더
    dir_ = f'/content/drive/MyDrive/코딩공부/dacon_project/model' # 모델이 저장될 경로
    pt = 'test_model' #학습된 모델 이름
    max_len = 100 # 최대 시퀀스 길이

    start_lr = 1e-5
    max_lr = 1e-4
    min_lr = 1e-6

    num_workes = 8
    seed = 42
    scheduler = None #학습률 스케줄러 지정

date_dir = './'
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) # gpu연결 성공

def set_seeds(seed = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.determinisitc = True #Ture로 설정하여 결과의 일관성 유지
    torch.backends.cudnn.benchmark = False #False로 설정하면 성능보다는 재현성을 우선시 함

set_seeds(seed = args.seed)

In [ ]:
# 현재 옵티마이저에서 사용중인 학습률을 반환
## 모니터링, 로그에 기록하기 위함. 학습률이 동적으로 변경될 때, 현재 적용되고 있는 학습률 값을 확인
def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
        lr += [ param_group['lr'] ] #리스트 형식으로 계속 저장이 되기 때문에, []로 한번 감싸줘야함.

    assert(len(lr)==1) #디버깅 툴 1이 아닐경우 type error발생
    lr = lr[0]

    return lr


def load_data():
    train = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')
    test = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/test.csv')

    #필요한 열만 추출
    train = train[['제목', '키워드']]
    test = test['제목']

    #
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=5, random_state = 42, shuffle = True)
    train['fold'] = -1
    for n_fold, (_, v_idx) in enumerate(skf.split(train, train['topic_idx'])):
        #split(train, train['topic_idx])는 topic_idx에 따라 클래스 비율을 유지하면서 k개의 폴드로 나눔 => 학습 인덱스와 검증 인덱스를 반환 함
        # n_fold는 현재 폴드 번호를 나타냄 (enumerate)를 통해서 0부터 시작하여 폴드 수만큼 증가함
        # (_, v_idx)는 학습 인덱스와 검증 인덱스를 반환함(학습 인덱스는 사용하지 않음)
        train.loc[v_idx, 'fold'] = n_fold
        # .loc는 .loc[행 인덱스 선택기, 열 이름 선택기] => v_idx에 해당하는 열의 fold에 n_fold 값을 넣어줌 (정한 행의 열 값에 값을 넣는 작업)
        # 위 작업을 통해서 -1로 설정되어 있던 fold에 n_fold값을 넣어줌(랜덤으로 정해진 그룹 fold번호)
    train['id'] = [x for x in range(len(train))] #유니크 번호 값 줌

    return train, test


In [ ]:
import logging #로그 메시지를 기록하기 위한 라이브러리
import unicodedata #유니코드 문자 데이터베이스 접근을 위한 모듈
from shutil import copyfile # 파일 복사

from transformers import PreTrainedTokenizer #Hugging Face의 transformers 라이브러리에서 사전 학습된 토크나이저의 기본 클래스

logger = logging.getLogger(__name__) #현재 모듈에 대한 로거를 생성 (logger 파일 만들어서 로그 정리)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

# 토크나이저에 필요한 파일들의 이름 지정
PRETRAINED_VOCAB_FILES_MAP = {
    # SentencePiece 모델 파일 이름
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    # 단어 집합을 텍스트 파일로 저장한 파일의 이름
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

# 사전 학습된 모델의 최대 시퀀스 길이(위치 임베딩 크기)
PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

# 각 모델에 대한 초기 설정값 지정(do_lower_case를 False로 설정하여 대소문자 구분)
PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁' #SentencePiece에서 사용하는 언더라인 문자로, 서브워드 토큰화에서 단어의 시작을 나타냄.

class KoBertTokenizer(PreTrainedTokenizer):
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file, # SentencePiece 모델 파일 경로
            vocab_txt,# 단어 집합을 저장한 텍스트 파일 경로
            do_lower_case = False, # 대소문자 구분 여부
            remove_sapce = True, #공백 제거 여부
            keep_accents = False, # 악센트 유지 여부
            unk_token = "[UNK]", # 아래 5개의 경우 특수 토큰 지정
            sep_token = "[SEP]",
            pad_token = "[PAD]",
            cls_token = "[CLS]",
            mask_token = "[MASK]",
            **kwrgs):
        super().__init__(
            unk_token = unk_token,
            sep_token = sep_token,
            pad_token = pad_token,
            cls_token = cls_token,
            mask_token = mask_token,
            **kwrgs
        )

        # vocab 파일 로드
        self.token2idx = dict() #토큰을 인덱스로 매핑
        self.idx2token = [] #인덱스를 토큰으로 매핑하는 리스트
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)
        # self.max_len_single_sentence = self.max_len - 2 # 특수 토큰 가져옴
        # self.max_len_sentences_pair = self.max_len - 3 #특수 토큰 가져옴

        '''
        SentencePiece는 구글에서 개발한 언어 독립적인 서브워드 토크나이저
        subword segmentation을 통해 희귀한 단어나 신조어도 효과적으로 처리 가능
        BPE(Byte Pair Encoding)와 유사하지만, 문장 수준이 아닌 단어 수준에서의 토큰화 수행
        복잡한 형태소 구조를 효과적으로 처리 가능
        공백 처리: 공백을 특수한 심볼(예: '_')로 처리하여, 단어 경계를 나타냄
        EX)
        text = "안녕하세요. 반갑습니다."
        tokens = sp.EncodeAsPieces(Text)
        print(tokens) => ['▁안녕하세요', '.', '▁반갑습니다', '.']
        '''


        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("KoBertTokenizer을 사용하기 위해서는 SentencePiece를 설치해야 합니다. 'pip install sentencepiece' ")

        # 인스턴스 변수 설정
        self.do_lower_case = do_lower_case
        self.remove_space = remove_sapce
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor() # 모델파일 로드
        self.sp_model.Load(vocab_file) #입력받은 파일 경로 저장



    @property # 메서드를 속성처럼 사용할 수 있게 해줌.
    # 이를 통해 클래스의 속성에 접근하는 것처럼 메서드를 호출할  수 있음 => KoBertTokenizer().vocab_size 의 형식으로 사용할 수 있음 KoBertTokenizer().vocab_size()가 아님
    # 토크나이저의 단어 집합 크기를 반환
    def vocab_size(self):
        return len(self.idx2token)
    '''
    직렬화: 객체의 상태를 저장하거나 전송할 수 있도록 바이트 스트림으로 변환
    역직렬화: 바이트 스트림을 원래의 객체로 복원하는 과정
    이를 통해 객체를 파일에 저장하거나 네트워크를 통해 전송할 수 있음
    '''
    def __getstate__(self):
    # 객체를 직렬화할 때, sp_model울 제외하고 상태를 저장합니다.
        state = self.__dict__.copy()
        state["sp_model"] = None #sp_model은 SentencePiece 모델 객체로, 직렬화할 수 없는 객체
        return state

    def __setstate__(self, d):
    # 객체를 역직렬화할 때, sp_model을 다시 로드
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("KoBertTokenizer을 사용하기 위해서는 SentencePiece를 설치해야 합니다. 'pip install sentencepiece' ")
        self.sp_model = spm.SentencePieceProcessor() # 모델파일 로드
        self.sp_model.Load(self.vocab_file) #입력받은 파일 경로 저장

    def preprocess_text(self, inputs):
        # 입력 텍스트 전처리
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"') # 따옴표 통일

        # 프랑스어, 스페인어의 악센트를 포함한 문자들의 경우를 전처리
        # NFKD를 정규화를 통해 문자를 분해
        # 원본 텍스트: école (프랑스어로 "학교")
        # NFKD 정규화 후: école (e와 조합 문자 ́로 분해)
        # 악센트 제거 후: ecole
        if not self.keep_accents:
            outputs = unicodedata.normilize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode = True, sample = False):
        text = self.preprocess(text) # 텍스트 전처리

        '''
        샘프링 기반 토큰화: 토큰화 과정에서 무작위성을 도입하여 여러 가능한 토큰화 결과 중 하나를 샘플링
        데이터 증강의 한 방법, 모델의 일반화 성능을 향상시키는데 도움이 됨.
        '''
        if not sample:
            pieces = self.sp_model.EncoderAsPieces(text)
        else:
            pieces = self.sp_model.SmapleEncodeAsPieces(text, 64, 0.1)
            #64는 후보 수 (nbest), 0.1은 온도(temperature) 값 => 데이터의 다양성을 높여 모델의 학습에 도움을 줌
            '''
            원본 문장: "자연어처리는 재미있습니다."
            샘플링 기반 토큰화 결과:
            첫 번째 샘플: ['▁자연어', '처리', '는', '▁재미있', '습니다', '.']
            두 번째 샘플: ['▁자연어', '처리', '는', '▁재미', '있습니다', '.']
            세 번째 샘플: ['▁자연어', '처리', '는', '▁재', '미있', '습니다', '.']
            '''
        new_pieces = []
        for piece in pieces:
            # 토큰의 길이가 2 이상이고, 마지막 글자가 쉼표(,)아묘 그 앞글자가 숫자인 경우
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                # 토큰의 마지막 글자(쉼표)를 제외한 부분을 가져온 후, 언더바 문자를 제거, 해당 부분을 다시 토큰화.
                # 결과적으로 쉼표를 제외한 부분을 재 토큰화
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    # 토큰을 인덱스로 변환. token2idx 딕셔너리에서 해당 토큰의 인덱스를 가져오며, 없을 경우 [UNK] 토큰의 인덱스 사용
    def _convert_token_to_id(self, token):
        # token2dix.get.(a,b)에서 token2dix에 a 값이 존재하면 a / 없으면 b를 반환
        return self.token2idx.get(token, self.token2idx[self.unk_token]) #[unk]토큰은 사전에 없는 토큰을 대체하는 특별한 토큰


    # 인덱스를 토큰으로 변환.
    def _convert_id_to_token(self, index, return_unicode = True):
        return self.idx2token[index]

    # 토큰의 리스트를 하나의 문자열로 변환. 토큰들을 합친 후, 언더라인 문자를 공백으로 대체하고 양쪽 공백을 제거
    def convert_tokens_to_string(self, tokens):
        '''
        1. "".jjoun(tokens): 토큰 리스트를 하나의 문자열로 결합
            ex) ['_Hello', 'world', '!'] => '_Helloworld!'
        2. 언더바 문자를 공백으로 대체
            .replace(SPIECE_UNDERLINE, " "): '_'문자를 공백으로 변경
            eX) '_HelloWorld!' => ' HelloWorld!'
        3. 양쪽 공백 제거:
            .strip(): 문자열 양쪽의 공백 제거
            ex) ' HelloWorld! ' => 'HelloWorld!'
        토큰화된 결과를 다시 하나의 문자열로 합쳐서 사람이 읽을 수 있는 문장 형태로 복원
        '''
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    # 모델 입력에 필요한 특수 토큰을 추가하여 입력 시퀀스를 생성함.
    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1 = None):
        '''
        입력 시퀀스 또는 시퀀스 쌍에 대해 특수 토큰을 추가하여 모델 입력을 생성합니다.
        BERT 시퀀스 형식:
            단일 시퀀스: [CLS] X [SEP]
            시퀀스 쌍: [CLS] A [SEP] B [SEP]
        token_ids_0: 첫 번째 시퀀시의 토큰 ID 리스트
        token_ids_1: 두 번째 시퀀스의 토큰 id 리스트
        self.cls_token_id: [CLS] 토큰의 ID
        self.sep_token_id: [SEP] 토큰의 ID
        [CLS]: 문장의 시작을 알리는 토큰
        [SEP]: 시퀀스 간의 구분을 나타내는 토큰
        '''
        if token_ids_1 is None:
            '''
            입력 시퀀스 형식: [CLS] token_ids_0 [SEP]
            token_ids_0 : [1012, 2345, 675]
            self.cls_token_id: 101
            self.sep_token_id: 102
            결과: [101, 1012, 2345, 675, 102]
            '''
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        '''
        입력 시퀀스 형식: [CLS] token_ids_0 [SEP] token_ids_1 [SEP]
        token_ids_0 : [1012, 2345, 675]
        token_ids_1 : [2001, 5432]
        self.cls_token_id: 101
        self.sep_token_id: 102
        결과: [101, 1012, 2345, 675, 102, 2001, 5432, 102]
        '''
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        '''
        텍스트 분류: 단일 시퀀스를 입력으로 받아 분류하는 경우(텍스트 분류)
            입력: [CLS] 문장 [SEP]
        문장 쌍 분류: 두 문장이 주어졌을 때 관계를 분류하는 경우(자연어 추론)
            입력: [CLS] 문장1 [SEP] 문장2 [SEP]
        '''
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1 = None, already_has_special_tokens = False):
        '''
        입력 시퀀스에서 특수 토큰의 위치를 나타내는 마스크를 생성
        Args:
            token_ids_0: 특수 토큰이 포함되지 않은 첫 번째 시퀀스의 토큰 ID 리스트
            token_ids_1: 특수 토큰이 포함되지 않은 두 번째 시퀀스의 토큰 ID 리스트
            already_has_special_tokens: 특수 토큰이 이미 포함된 시퀀스인지 여부
        Returns:
            0과 1로 이루어진 리스트: 1은 특수토큰, 0은 일반 토큰을 나타냄
        '''
        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError("이미 특수 토큰이 포함된 경우 두 번째 시퀀스를 제공하면 안됩니다")
            '''
            token_ids_1이 제공되면 에러를 발생. 이미 특수 토큰이 포함된 경우 시퀀스 쌍을 처리하지 않음
            token_ids_0:
                해당 토큰이 [CLS] 또는 [SEP] 토큰의 ID와 같으면 1을 반환(특수 토큰)
                그렇지 않으면 0을 반환(일반 토큰)
            '''
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            '''
            1은 특수토큰, 0은 일반 토큰
            [1, 0, 0 ..... 1]
            마스킹 언어 모델이나 다음 토큰 예측 등에서 특수 토큰은 손실 계산에서 제외하여야 함.
            '''
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        '''
        [1, 0, 0, 1, 0, 0, ... , 1]
        '''
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1 = None):
        '''
        token type ID 생성: Bert 모델은 두 개의 시퀀스(문장 쌍)를 입력으로 받을 떄, 각 시퀀스가 어떤 문장에 속하는지를 나타내기 위해 token_type_ids를 사용.
        이 메서드는 두 시퀀스의 토큰 타입 ID를 생성
        시퀀스 구분:token_type_ids는 각 토큰이 첫 번째 시퀀스에 속하는지, 두 번째 시퀀스에 속하는지를 구분하는 역할. 이를 통해서 BERT는 입력된 문장 쌍을 이해하고 처리 가능
        '''
        sep = [self.sep_token_id] # 토큰 ID. 두 시퀀스를 구분하는데 사용
        cls = [self.cls_token_id] # 토큰 ID. BERT 모델의 입력 시퀀스의 시작을 나타냄
        if token_ids_1 is None:
            '''
            [CLS] 토큰과 [SEP] 토큰을 포함한 전체 시퀀스의 길이 계산. 해당 길이만큼 0으로 채워진 리스트 반환.
            단일 시퀀스 입력의 경우 모든 토큰 타입 ID가 0 => 모든 토큰이 첫번째 시퀀스에 속함
            [0, 0, 0, 0, 0, .... 0, 0, 0]
            '''
            return len(cls + token_ids_0 + sep) * [0]
        '''
        첫 번째 시퀀스의 길이에 맞춰서 0으로 채워진 리스트 생성
        두 번째 시퀀스의 길이에 맞춰서 1으로 채워진 리스트 생성
        두 리스트를 결합하여 전체 token_type_ids 생성
        [0, 0, 0, 0, 0, .... 1, 1, 1]
        '''
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        '''
        어휘 사전 저장: 토크나이저의 어휘 사전을 지정된 디렉토리에 저장
        SentencePiece 모델과  Vocab.txt 파일 저장: 두 가지 주요 파일을 저장하여 나중에 토크나이저를 다시 로드하거나 다른 환경에서 사용할 수 있도록 함.
        '''
        # 저장 디렉토리 확인
        if not os.path.isdir(save_directory):
            logger.error("어휘 경로 path ({})가 존재하지 않습니다." .format(save_directory))
            return

        # SentencePiece 모델 파일 경로 저장
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            # 경로가 동일하지 않으면 vocab_file을 out_vocab_model로 복사
            copyfile(self.vocab_file, out_vocab_model)

        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding = "utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key = lambda kv: kv[1]):
                if index != token_index: #예상 인덱스와 실제 토큰 인덱스가 일치 하지 않으면 경고 로그 출력(인덱스가 연속적으로 맞는지 확인하는 작업)
                    logger.warning(
                        "어휘 파일이 손상되었습니다. 토큰 ID가 순차적이지 않습니다. ({})".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1
        return out_vocab_model, out_vocab_txt

In [ ]:
# sent를 토크나이저를 사용하여 토큰화하고, BERT 모델에 필요한 입력형식으로 변환
def bert_tokenizer(sent, MAX_LEN, tokenizer):
    '''"
    sent: 문자열 형태의 입력 문장
    MAX_LEN: 최대 시퀀스 길이
    tokenizer: 토크나이저 객체(ex - KoBertTokenizer, AutoTokenizer 등)
    '''
    # encode_plus는 입력된 텍스트를 토큰화하고 인코딩한 결과를 딕셔너리 형태로 반환
    encoded_dict = tokenizer.encode_plus(
        text = sent, # 입력 문장
        add_special_tokens = True, # 특별 토큰(CLS, SEP 등 추가)
        max_length = MAX_LEN, # 최대 시퀀스 길이 설정
        padding='max_length', # 최대 길이에 맞추어 패딩을 추가
        return_attention_mask = True, # 어텐션 마스크를 반환
        return_token_type_ids=True, # 토큰 타입 ID를 반환
        truncation = True # 최대 길이를 초과하는 시퀀스를 자름
    )

    # inpud_id, attention_mask를 따로 빼서 작성하는 이유는 필요한 정보만 선택적으로 사용하기 위해서 (코드 명확성 / 가독성)
    input_id = encoded_dict['input_ids'] #토큰을 인덱스로 변환한 리스트
    attention_mask = encoded_dict['attention_mask'] # 패딩된 위치를 고려한 어텐션 마크스(1은 실제 토큰, 0은 패딩)
    token_type_id = encoded_dict.get('token_type_ids') # 문장구분을 위한 토큰 타입 IDs. 문장 쌍 입력 시에 사용  (여기서는 단일 문장을 처리하므로 사용하지 않음) => token_type_id는 0으로 설정
    # token_type_id = 0

    return input_id, attention_mask, token_type_id

# 학습 데이터를 전처리하여 모델 학습에 사용할 수 있는 형태로 변환. 전처리된 데이터를 저장
def preprocessing_train():
    pt = args.pt # 모델 타입 설정

    # pt 변수에 'kobert' 문자열이 포함되어 있으면 KobertTokenizer 사용
    # tokenizer 객체는 입력된 텍스트를 모델이 이해할 수 있는 형식으로 변환
    # if 'kobert' in pt:
    #     tokenizer = KoBertTokenizer.from_pretrained(pt, cache_dir = 'bert_ckpt', do_lower_case = False) #cache_dir = 'bert_ckpt'를 통해 캐티 디렉토리를 지정 => 파일의 저장 위치 제어 및 파일 관리 유용
        # print('load kobert')
    # 만약 없다면, AutoTokenizer를 사용하여 지정된 pre-trained 모델의 토크나이저를 로드
    # else:
    # if문에 걸리면 에러가 발생하니, 주석처리 함
    tokenizer = AutoTokenizer.from_pretrained(pt, use_fast=False)
#AutoTokenizer(허깅페이스에서 제공하는 범용 토크나이저) 메서드를 통해 모델 이름을 지정하면 해당 모델에 맞는 토크나이저를 자동으로 로드함.

    MAX_LEN = args.max_len # 최대 시퀀스 길이 가져옴
    train = pd.read_csv('/content/drive/MyDrive/example/train_data.csv')
    train = train[['title', 'topic_idx']]

    # 레이블 인코딩 추가
    le = LabelEncoder()
    train['topic_idx'] = le.fit_transform(train['topic_idx'])

    # 레이블 인코더 저장
    os.makedirs(f'/content/drive/MyDrive/example/data/{pt}/', exist_ok=True)
    with open(f'/content/drive/MyDrive/example/data/{pt}/label_encoder.pkl', 'wb') as f:
      pickle.dump(le, f)

    # 토큰화된 입력과 레이블을 저장할 빈 리스트를 생성
    input_ids = []
    attention_masks = []
    token_type_ids= []
    train_data_labels = []

    for train_sent, train_label in tqdm.tqdm(zip(train['title'], train['topic_idx'])):
        try:
            # bert_tokenizer 함수를 호출하여 토큰화 및 인코딩을 수행
            input_id, attention_mask,_ = bert_tokenizer(train_sent, MAX_LEN=MAX_LEN, tokenizer=tokenizer)

            # 반환된 input_id, attention_mask, token_type_id를 리스트에 추가 (token_type_id는 여기서는 제외)
            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(0) # 단일 문장이니 0만 추가
            ##################################
            train_data_labels.append(train_label)

        except Exception as e:
            print(e)
            pass # 에러나면 print 하고 넘어감

    # 리스트를 넘파이 배열로 변환하여 모델 입력에 사용할 수 있도록
    train_input_ids = np.array(input_ids, dtype=int)
    train_attention_masks = np.array(attention_masks, dtype=int)
    train_token_type_ids = np.array(token_type_ids, dtype=int)
    ########################################################
    train_inputs = (train_input_ids, train_attention_masks, train_token_type_ids)
    train_labels = np.asarray(train_data_labels, dtype=np.int32)

    #save
    train_data = {}

    # 전처리 된 데이터를 하나의 딕셔너리에 저장
    train_data['input_ids'] = train_input_ids
    train_data['attention_mask'] = train_attention_masks
    train_data['token_type_ids'] = train_token_type_ids
    train_data['targets'] = np.asarray(train_data_labels, dtype = np.int32)

    # 모델 타입에 따라 저장할 디렉토리를 생성 / 피클 파일로 저장 / pickle.HIGHEST_PROTOCL은 가장 최신 프로토콜 버전으로 저장
    os.makedirs(f'/content/drive/MyDrive/example/data/{pt}/', exist_ok=True)
    with open(f'/content/drive/MyDrive/example/data/{pt}/train_data_{MAX_LEN}.pickle', 'wb') as f:
        pickle.dump(train_data, f, pickle.HIGHEST_PROTOCOL)


def preprocessing_test():
    pt = args.pt
    tokenizer = AutoTokenizer.from_pretrained(pt, use_fast=False)
    print(f'Loaded tokenizer from {pt}')

    MAX_LEN = args.max_len

    test = pd.read_csv('/content/drive/MyDrive/대회/gbt 대회/test_data.csv')
    test = test[['title']]

    input_ids = []
    attention_masks = []
    token_type_ids = []

    for idx, test_sent in tqdm.tqdm(enumerate(test['title']), total=len(test)):
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAX_LEN=MAX_LEN, tokenizer=tokenizer)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        if token_type_id is not None:
            token_type_ids.append(token_type_id)

    test_input_ids = np.array(input_ids, dtype=int)
    test_attention_masks = np.array(attention_masks, dtype=int)
    if token_type_ids:
        test_token_type_ids = np.array(token_type_ids, dtype=int)
    else:
        test_token_type_ids = None

    test_data = {
        'input_ids': test_input_ids,
        'attention_mask': test_attention_masks
    }
    if test_token_type_ids is not None:
        test_data['token_type_ids'] = test_token_type_ids

    os.makedirs(f'/content/drive/MyDrive/example/data/{pt}/', exist_ok=True)
    with open(f'/content/drive/MyDrive/example/data/{pt}/test_data_{MAX_LEN}.pickle', 'wb') as f:
        pickle.dump(test_data, f, pickle.HIGHEST_PROTOCOL)

    print(f"처리된 테스트 샘플 수: {len(input_ids)}")



In [ ]:
'''
여러개의 사전 학습된 NLP(BERT 기반 모델)을 사용하여 학습 및 테스트 데이터를 전처리하는 과정.
반복문을 활용하여 각 모델에 맞는 전처리를 자동으로 실행.
'''
# zip 험수: 두 개 이상의 리스트를 병렬로 묶어서 동시세 순회 (모델 이름 리스트와 최대 길이 리스트 두 개를 병렬로 순회)
# [33,33,33,33,33,33,28]는 최대 길이 리스트
# max_len이 28~33으로 짧게 들어간 이유는 뉴스 기사 제목은 짧은 문장으로 구성되어서 28~33 정도로 설정
# 모델별 최적화: 특정 시퀀스 길이에서 최적의 성능을 발휘할 수 있도록 학습되어 있음
for pt, max_len in zip(['monologg/kobert','klue/roberta-base','klue/roberta-small','klue/roberta-large','xlm-roberta-large',
           'bert-base-multilingual-uncased', 'klue/roberta-large'],[33,33,33,33,33,33,28]):
    '''
    KoBERT: 한국어에 특화된 Bert 모델
    klue/Roberta: KLUE(Korean Language Understanding Evaluation) 벤치마크를 기반으로 학습된 RoBERTa 시리즈
    xlm-roberta-large: 다국어 지원을 위한 RoBERTa 모델
    bert-base-multilingual-uncased: 104개 언어를 지원하는 BERT 모델로, 대소문자를 구분하지 않는 버전(uncased)
    '''
    args.max_len = max_len
    args.pt = pt

    # 전처리: 학습데이터의 각 문장을 토큰화 한 후, 필요한 특수 토큰(cls, spe), 패딩, 어텐션 마스크 등을 추가
    preprocessing_train()
    preprocessing_test()

    print(f'{args.pt} 모델 전처리 완료')

In [ ]:
# DatsET은 PyTorch에서 데이터셋을 다루기 위한 기본 클래스
class KobertDataSet(Dataset):
    '''
    Dataset = 데이터셋 객체 초기화
    위의 preprocessing_train / preprocessing_test 함수에서 생성된 피클 파일을 로드한 후의 데이터
    'input_ids' : 토큰화된 입력의 인덱스 리스트
    'attention_mask': 어텐션 마스크
    'token_type_ids': 토큰 타입 ID
    'targets': 레이블
    '''
    # test = False 기본값 -> 훈련 데이터로 본다는 것
    def __init__(self, data, test = False):
        self.data = data
        self.test = test

    # 데이터셋의 샘플 수 반환
    def __len__(self):
        # 배열의 첫번째 차원 =-> 데이터의 샘플 수
        return self.data['input_ids'].shape[0]

    # 특정 인덱스에 해당하는 샘플을 반환
    def __getitem__(self, idx):
        '''
        테스트 데이터
        ids: 입력 토큰의 인덱스
        mask: 어텐션 마스크
        token_type_ids: 토큰 타입 ID
        ------------------------------------------------
        targets: 레이블 (학습 데이터일 경우만 추가)
        '''
        # 특정 인덱스에 해당하는 inpud_ids, attention_mask, token_type_ids를 가져와서 Pytorch 텐서로 변환 (정수형)
        ids = torch.tensor(self.data['input_ids'][idx], dtype=torch.long)
        mask = torch.tensor(self.data['attention_mask'][idx], dtype=torch.long)

        if 'token_type_ids' in self.data:
            token_type_ids = torch.tensor(self.data['token_type_ids'][idx], dtype=torch.long)
        else:
            token_type_ids = None

        if self.test:
            sample = {'ids': ids, 'mask': mask}
            if token_type_ids is not None:
                sample['token_type_ids'] = token_type_ids
            return sample
        else:
            target = torch.tensor(self.data['targets'][idx], dtype=torch.long)
            sample = {'ids': ids, 'mask': mask, 'targets': target}
            if token_type_ids is not None:
                sample['token_type_ids'] = token_type_ids
            return sample

In [ ]:
# ------------------------
#  scheduler
# ------------------------

# 모델 성능 평가
def do_valid(net, valid_loader):

    val_loss = 0  # 검증 데이터 손실 값
    target_lst = []  # 실제 레이블 저장
    pred_lst = []  # 예측 레이블 저장
    logit = []  # 예측 로짓 저장 (출력값(로짓) - 각 클래스에 대한 비정규화된 점수로 소프트맥스 함수를 적용하기 전의 값) -> 손실 계산 시 사용
    loss_fn = nn.CrossEntropyLoss()  # 손실 함수로 CrossEntropyLoss 사용

    net.eval()  # 모델을 평가모드로 전환 (드롭아웃, 배치 정규화 등이 작동하지 않음)
    start_timer = timer()

    # 입력 데이터와 레이블을 GPU로 이동
    for t, data in enumerate(tqdm.tqdm(valid_loader)):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        tokentype = data['token_type_ids'].to(device)
        target = data['targets'].to(device)

        # torch.no_grad()를 사용하여 그레디언트 계산 비활성화
        with torch.no_grad():
            # 혼합 정밀도 모드로 전환하여 연산 속도 향상
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(ids, mask)
                    output = output[0]

                    # loss
                    loss = loss_fn(output, target)
            # 혼합 정밀도 모드가 아닌 경우 일반적인 방법으로 수행
            else:
                output = net(ids, mask)
                loss = loss_fn(output, target)

            val_loss += loss.item()  # 손실을 배치 단위로 합산, 텐서에서 값 추출
            target_lst.extend(target.detach().cpu().numpy())  # 레이블 텐서를 그레디언트 그래프에서 분리하고 cpu로 이동 후 Numpy 배열로 변환
            pred_lst.extend(output.argmax(dim=1).tolist())  # 로짓 중 가장 높은 값의 인덱스를 예측값으로 저장
            logit.extend(output.tolist())  # 로짓 저장

    # 배치 처리 후 평균 손실 및 성능 계산
    val_mean_loss = val_loss / len(valid_loader)  # 전체 배치의 손실을 배치 수로 나누어 평균 손실 계산
    validation_score = f1_score(y_true=target_lst, y_pred=pred_lst, average='macro')  # F1 스코어 계산
    validation_acc = accuracy_score(y_true=target_lst, y_pred=pred_lst)  # 정확도 계산

    return val_mean_loss, validation_score, validation_acc, logit


# 위 do_valid 함수와 비슷
def do_predict(net, valid_loader):

    val_loss = 0
    pred_lst = []
    logit=[]
    net.eval()
    for t, data in enumerate(tqdm.tqdm(valid_loader)):
        ids  = data['ids'].to(device)
        mask  = data['mask'].to(device)
        tokentype = data['token_type_ids'].to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(ids, mask)[0]

            else:
                output = net(ids, mask)

            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())

    return pred_lst,logit

# 모델 학습. 다중 폴드 교차 검증 수행
def run_train(folds=3):
    # 모델 저장 디렉토리 생성
    out_dir = args.dir_+ f'/fold{args.fold}/{args.exp_name}/'
    os.makedirs(out_dir, exist_ok=True)

    # load dataset
    # 각 모델별 전처리된 데이터 불러옴
    train, test = load_data()
    with open(f'/content/drive/MyDrive/example/data/{args.pt}/train_data_{args.max_len}.pickle', 'rb') as f:
        train_data = pickle.load(f)
    with open(f'/content/drive/MyDrive/example/data/{args.pt}/test_data_{args.max_len}.pickle', 'rb') as f:
        test_data = pickle.load(f)

    # split fold
    # 특정 폴드만 선택하여 학습함. 나머지는 건너 뜀
    for n_fold in range(5):
        if n_fold != folds:
            print(f'{n_fold} fold pass'+'\n')
            continue

        # 디버그 모드시 데이터를 1000개로 전환
        if args.debug:
            train = train.sample(1000).copy()

        trn_idx = train[train['fold']!=n_fold]['id'].values
        val_idx = train[train['fold']==n_fold]['id'].values


        train_dict = {'input_ids' : train_data['input_ids'][trn_idx] , 'attention_mask' : train_data['attention_mask'][trn_idx] ,
                      'token_type_ids' : train_data['token_type_ids'][trn_idx], 'targets' : train_data['targets'][trn_idx]}
        val_dict = {'input_ids' : train_data['input_ids'][val_idx] , 'attention_mask' : train_data['attention_mask'][val_idx] ,
                      'token_type_ids' : train_data['token_type_ids'][val_idx], 'targets' : train_data['targets'][val_idx]}

        ## dataset ------------------------------------
        train_dataset = KobertDataSet(data = train_dict)
        valid_dataset = KobertDataSet(data = val_dict)
        trainloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size,
                                 num_workers=8, shuffle=True, pin_memory=True)
        validloader = DataLoader(dataset=valid_dataset, batch_size=args.batch_size,
                                 num_workers=8, shuffle=False, pin_memory=True)

        ## net ----------------------------------------
        scaler = amp.GradScaler()
        # 분류 라벨 설정
        if 'xlm-roberta' in args.pt:
            net = XLMRobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

        elif 'klue/roberta' in args.pt:
            net = RobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)
        else:
            net = BertForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

        net.to(device)
        if len(args.gpu)>1:
            net = nn.DataParallel(net) #여러 GPU를 사용하는 경우, 모델을 병렬로 분산

        # ------------------------
        # loss
        # ------------------------
        loss_fn = nn.CrossEntropyLoss()

        # ------------------------
        #  Optimizer
        # ------------------------
        # RAddam optimizer을 통해서 학습 초반의 불안정성을 완화. Lookahead: RAdam 위에 Lookahead 옵티마이저를 적용하여 학습 속도와 성능 향상
        # filter(lambda p: p.requires_grad,net.parameters()): 학습 가능한 파라미터만 옵티마이저에 전달
        optimizer = optim.Lookahead(optim.RAdam(filter(lambda p: p.requires_grad,net.parameters()), lr=args.start_lr), alpha=0.5, k=5)

        # get_linear_schedule_with_warmup: 학습률을 서서히 증가시킨 후 선형적으로 감소
        # num_warmip_steps: 초기 학습률을 증가시키는 단계 수. 0일 경우 워밍업 단계 없이 바로 선형 감소 시작
        # num_training_steps = len(trainloader)*args.epochs: 전체 학습 단계 수
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = len(trainloader)*args.epochs)

        # ----
        start_timer = timer()
        best_score = 0

        for epoch in range(1, args.epochs+1):
            train_loss = 0
            valid_loss = 0

            target_lst = []
            pred_lst = []
            lr = get_learning_rate(optimizer)
            print(f'-------------------')
            print(f'{epoch}epoch start')
            print(f'-------------------'+'\n')
            print(f'learning rate : {lr : .6f}')
            for t, data in enumerate(tqdm.tqdm(trainloader)):

                # one iteration update  -------------
                ids  = data['ids'].to(device)
                mask  = data['mask'].to(device)
                tokentype = data['token_type_ids'].to(device)
                target = data['targets'].to(device)

                # ------------
                # 모델을 학습모드로 전환
                net.train()
                optimizer.zero_grad()


                if args.amp:
                    with amp.autocast():
                        # output
                        output = net(ids, mask)
                        output = output[0]

                        # loss
                        loss = loss_fn(output, target)
                        train_loss += loss


                    scaler.scale(loss).backward() # 손실을 스케일링하여 역전파 수행
                    scaler.step(optimizer) # 옵티마이저 업데이트
                    scaler.update() # 스케일러 업데이트

                else:
                    # output
                    output = net(ids, mask)

                    # loss
                    loss = loss_fn(output, target)
                    train_loss += loss

                    # update
                    loss.backward()
                    optimizer.step()


                # for calculate f1 score
                target_lst.extend(target.detach().cpu().numpy()) # 실제 레이블 저장
                pred_lst.extend(output.argmax(dim=1).tolist()) # 예측 레이블 저장


                if scheduler is not None:
                    scheduler.step()  #스케줄러 있을 경우 업데이트
            train_loss = train_loss / len(trainloader)
            train_score = f1_score(y_true=target_lst, y_pred=pred_lst, average='macro')
            train_acc = accuracy_score(y_true=target_lst, y_pred=pred_lst)

            # validation
            valid_loss, valid_score, valid_acc, _ = do_valid(net, validloader)

            # 정확도 높으면 저장
            if valid_acc > best_score:
                best_score = valid_acc
                best_epoch = epoch
                best_loss = valid_loss

                torch.save(net.state_dict(), out_dir + f'/{folds}f_{epoch}e_{best_score:.4f}_s.pth')
                print('best model saved'+'\n')


            print(f'train loss : {train_loss:.4f}, train f1 score : {train_score : .4f}, train acc : {train_acc : .4f}'+'\n')
            print(f'valid loss : {valid_loss:.4f}, valid f1 score : {valid_score : .4f}, valid acc : {valid_acc : .4f}'+'\n')

        # loss, score, accuracy 출력
        print(f'best valid loss : {best_loss : .4f}'+'\n')
        print(f'best epoch : {best_epoch }'+'\n')
        print(f'best accuracy : {best_score : .4f}'+'\n')

def run_predict(model_path):
    # dataset
    # load
    with open(f'./data/{args.pt}/test_data_{args.max_len}.pickle', 'rb') as f:
        test_dict = pickle.load(f)

    print('test load')
    test_dataset = KobertDataSet(data = test_dict, test = True)
    testloader = DataLoader(dataset = test_dataset, batch_size = args.batch_size,
                            num_workers = 8, shuffle = False, pin_memory = True)
    print('set testloader')
    ## net
    scaler = amp.GradScaler()
    if 'xlm-roberta' in args.pt:
        net = XLMRobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

    elif 'klue/roberta' in args.pt:
        net = RobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)
    else:
        net = BertForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

    net.to(device)

    if len(args.gpu) > 1:
        net == nn.DataParallel(net)

    f = torch.load(model_path)
    net.load_state_dict(f, strict = True) # strict = True: 모델의 파라미터가 완벽하게 일치하지 않으면 오류 발생
    print('load saved models')
    # validation
    preds, logit = do_predict(net, testloader)

    print('complete predict')

    return preds, np.array(logit)

In [ ]:
# """5fold 전용"""
# if __name__ == '__main__':
#     for pt, max_len in zip(['monologg/kobert','klue/roberta-base','klue/roberta-small','klue/roberta-large','xlm-roberta-large',
#            'bert-base-multilingual-uncased'],[33,33,33,33,33,33,28]):

#         args.max_len = max_len
#         args.pt = pt
#         args.exp_name = str(args.pt) + '_' + str(args.max_len)

#         for i in [0,1,2,3,4]: # 5fold
#             run_train(folds=i)

In [ ]:
if __name__ == '__main__':
    # 테스트에 사용할 모델 파일 경로를 지정합니다.
    model_path = '/content/drive/MyDrive/example/saved_models/fold3/klue/roberta-large_33/0f_10e_0.4615_s.pth'

    # 테스트에 사용할 모델과 max_len을 설정합니다.
    args.pt = 'klue/roberta-large'
    args.max_len = 33

    # 테스트 데이터 전처리가 필요한 경우 수행합니다.
    preprocessing_test()

    # 예측 수행
    preds, logits = run_predict(model_path)

    # 결과 저장 또는 후처리
    # 예: 제출 파일 생성
    submission = pd.read_csv('/content/drive/MyDrive/example/sample_submission.csv')
    submission['분류'] = preds  # 또는 적절한 라벨 변환 적용

    # 파일 저장
    submission.to_csv(os.path.join('/content/drive/MyDrive/example', f'submission_{args.pt.replace("/", "_")}_{args.max_len}.csv'), index=False)
